# Test data investigation
There were drastic difference between the train/val and test metrics. Test set contrains 2 cases where one gives comparable results to the val set and the other one is much worse. Let's investigate the test set to understand the difference.

In [ ]:
import torchio as tio
import numpy as np
import matplotlib.pyplot as plt


def show_test_case(i: int, fold = 0, figsize=None):
    test_cases = ['706-005', '707-003']
    test_case = test_cases[i]
    subject_a = tio.Subject(
        image=tio.ScalarImage(f'/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/imagesTs/{test_case}_0000.nii.gz'),
        unet3d=tio.ScalarImage(f'/storage_bizon/naravich/nnUNet_Datasets/nnUNet_results/Dataset301_Calcium_OCT/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_{fold}_test/{test_case}.nii.gz'),
        unet2d=tio.ScalarImage(f'/storage_bizon/naravich/nnUNet_Datasets/nnUNet_results/Dataset301_Calcium_OCT/nnUNetTrainer__nnUNetPlans__2d/fold_{fold}_test/{test_case}.nii.gz'),
        gt=tio.ScalarImage(f'/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/labelsTs/{test_case}.nii.gz')

    )
    subject_a.plot(figsize=figsize)
    return subject_a

In [ ]:
show_test_case(0)

In [ ]:
subject_1 = show_test_case(1, figsize=(15, 15))

In [ ]:
np.unique(subject_1.gt.numpy())

## Let's check all the labels 


In [ ]:
from pathlib import Path


def show_all_labels(figsize=None, plot=True):
    train_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/imagesTr')
    train_label_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/labelsTr')
    test_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/imagesTs')
    test_label_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/labelsTs')
    train_cases = [case.stem[:-4] for case in train_label_dir.glob('*.nii.gz')]
    test_cases = [case.stem[:-4] for case in test_label_dir.glob('*.nii.gz')]
    case2path = {f'tr_{case}': train_label_dir/f'{case}.nii.gz' for case in train_cases}
    case2path.update({f'ts_{case}': test_label_dir / f'{case}.nii.gz' for case in test_cases})
    subject_a = tio.Subject(
        **{case: tio.LabelMap(path) for case, path in case2path.items()}
    )
    subject_a.plot(figsize=figsize)
    return subject_a


def show_all_images(figsize=None):
    train_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/imagesTr')
    train_label_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/labelsTr')
    test_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/imagesTs')
    test_label_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/labelsTs')
    train_cases = [case.stem[:-4] for case in train_label_dir.glob('*.nii.gz')]
    test_cases = [case.stem[:-4] for case in test_label_dir.glob('*.nii.gz')]
    case2path = {f'tr_{case}': train_dir/f'{case}_0000.nii.gz' for case in train_cases}
    case2path.update({f'ts_{case}': test_dir / f'{case}_0000.nii.gz' for case in test_cases})
    subject_a = tio.Subject(
        **{case: tio.ScalarImage(path) for case, path in case2path.items()}
    )
    subject_a.plot(figsize=figsize)
    return subject_a

def show_all_images(figsize=None):
    train_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/imagesTr')
    train_label_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/labelsTr')
    test_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/imagesTs')
    test_label_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/labelsTs')
    train_cases = [case.stem[:-4] for case in train_label_dir.glob('*.nii.gz')]
    test_cases = [case.stem[:-4] for case in test_label_dir.glob('*.nii.gz')]
    case2path = list((f'tr_{case}', train_dir/f'{case}_0000.nii.gz') for case in train_cases)
    case2path += list((f'ts_{case}', test_dir / f'{case}_0000.nii.gz') for case in test_cases)

    case2path_label = list((f'trl_{case}', train_label_dir/f'{case}.nii.gz') for case in train_cases)
    case2path_label += list((f'tsl_{case}', test_label_dir / f'{case}.nii.gz') for case in  test_cases)

    merged = dict()
    for image, label in zip(case2path, case2path_label):
        merged[image[0]] = tio.ScalarImage(image[1])
        merged[label[0]] = tio.LabelMap(label[1])
    subject_a = tio.Subject(
        **merged
    )
    subject_a.plot(figsize=figsize)
    return subject_a

In [ ]:
show_all_labels(figsize=(40, 20))

In [ ]:
show_all_images(figsize=(40, 20))


In [ ]:
show_all_images(figsize=(80, 20))

# Fold_3 Investigation
- Fold 3 has the worst performance in the test set. Let's investigate the test set to understand the difference.

In [ ]:
def show_fold_3(figsize=None):
    prediction_3d_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_results/Dataset301_Calcium_OCT/nnUNetTrainer__nnUNetPlans__3d_fullres/fold_3/validation/')
    prediction_2d_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_results/Dataset301_Calcium_OCT/nnUNetTrainer__nnUNetPlans__2d/fold_3/validation/')
    train_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/imagesTr')
    train_label_dir = Path('/storage_bizon/naravich/nnUNet_Datasets/nnUNet_raw/Dataset301_Calcium_OCT/labelsTr')
    subject_a = tio.Subject(
        image=tio.ScalarImage(train_dir / '701-013_0000.nii.gz'),
        unet3d=tio.LabelMap(prediction_3d_dir / '701-013.nii.gz'),
        unet2d=tio.LabelMap(prediction_2d_dir / '701-013.nii.gz'),
        gt=tio.LabelMap(train_label_dir / '701-013.nii.gz')

    )
    subject_a.plot(figsize=figsize)
    return subject_a
show_fold_3(figsize=(15, 15))